In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy

plink command for GRS calculation:

plink2  --out il6_12var_grs --pfile /cluster/UKB_IMP/IMP_v3/chr7 --score snps_12var.txt 1 4 6 cols=+scoresums no-mean-imputation ignore-dup-ids

1st column – rsID
4th column – effect allele
6th column - beta estimate

In [2]:
# read files for different genetic instruments calculated by plink 


file1 = '../il6_12var_grs.sscore'
file2 = '../il6_8var_grs.sscore'
file3 = '../il6_3var_grs.sscore'

df1 = pd.read_table(file1)
df2 = pd.read_table(file2)
df3 = pd.read_table(file3)

merged_df = df1[['IID','SCORE1_SUM']].merge(df2[['IID','SCORE1_SUM']], on='IID', suffixes=('_12var', '_8var'))
merged_df = merged_df.merge(df3[['IID','SCORE1_SUM']], on ='IID')

In [3]:
merged_df = merged_df.rename(columns={'SCORE1_SUM':'SCORE1_SUM_3var'})

In [4]:
df = pd.read_table('descr_df_with_CRP_and_GRS.tsv')

In [5]:
df = df.merge(merged_df,on ='IID')

In [6]:
df_ = deepcopy(df)

In [8]:

def bootstrap_percent_decrease(top, bottom, n_bootstrap=10000, seed=42):
    """
    Function for percent decrease estimation with bootstrap
    """
    np.random.seed(seed)
    perc_decreases = []
    for _ in range(n_bootstrap):
        top_sample = np.random.choice(top, size=len(top), replace=True)
        bottom_sample = np.random.choice(bottom, size=len(bottom), replace=True)
        top_median = np.median(top_sample)
        bottom_median = np.median(bottom_sample)
        perc_decrease = 100 * (top_median - bottom_median) / top_median
        perc_decreases.append(perc_decrease)
    median_decrease = 100 * (np.median(top) - np.median(bottom)) / np.median(top)
    ci_low = np.percentile(perc_decreases, 2.5)
    ci_high = np.percentile(perc_decreases, 97.5)
    return median_decrease, ci_low, ci_high


In [9]:
def decrease_median(dataframe, score):
    
    """
    Function to calculate median decrease in CRP levels
    
    """
    col_name = f'SCORE1_SUM_{score}var'
    thresholds = {
    '1': np.percentile(dataframe[col_name], 1),
    '99': np.percentile(dataframe[col_name], 99),
    '5': np.percentile(dataframe[col_name], 5),
    '95': np.percentile(dataframe[col_name], 95),
    '10': np.percentile(dataframe[col_name], 10),
    '90': np.percentile(dataframe[col_name], 90),
    '25': np.percentile(dataframe[col_name], 25),
    '75': np.percentile(dataframe[col_name], 75)
    }

# Define comparisons
    comparisons = [
        ('Top 1%', dataframe[dataframe[col_name] >= thresholds['99']]['CRP'],
         'Bottom 1%', dataframe[dataframe[col_name] <= thresholds['1']]['CRP']),

        ('Top 5%', dataframe[dataframe[col_name] >= thresholds['95']]['CRP'],
         'Bottom 5%', dataframe[dataframe[col_name] <= thresholds['5']]['CRP']),

        ('Top 10%', dataframe[dataframe[col_name] >= thresholds['90']]['CRP'],
         'Bottom 10%', dataframe[dataframe[col_name] <= thresholds['10']]['CRP']),

        ('Top 25%', dataframe[dataframe[col_name] >= thresholds['75']]['CRP'],
         'Bottom 25%', dataframe[dataframe[col_name] <= thresholds['25']]['CRP'])
    ]

    # Compute percent decreases
    results = []
    for top_label, top_crp, bottom_label, bottom_crp in comparisons:
        if len(top_crp) > 0 and len(bottom_crp) > 0:
            percent_decrease, ci_low, ci_high = bootstrap_percent_decrease(top_crp.values, bottom_crp.values)
            results.append({
                'Comparison': f'{top_label} vs {bottom_label}',
                'Percent Decrease in Median CRP': percent_decrease,
                '95% CI Lower': ci_low,
                '95% CI Upper': ci_high,
                'Top n': len(top_crp),
                'Bottom n': len(bottom_crp)
            })
    return pd.DataFrame(results)

In [10]:
### Generate statistics for every GRS: SCORE1_SUM_12var,SCORE1_SUM_8var, SCORE1_SUM_3var
res_12var = decrease_median(df_,'12')

In [11]:
### Create a readable output
res_12var['median_estimate'] = np.round(res_12var['Percent Decrease in Median CRP'],2).astype(str) + ' [' \
+np.round(res_12var['95% CI Lower'],2).astype(str) +'-'+ np.round(res_12var['95% CI Upper'],2).astype(str) + ']'

In [12]:
res_12var

,Comparison,Percent Decrease in Median CRP,95% CI Lower,95% CI Upper,Top n,Bottom n,median_estimate
0,Top 1% vs Bottom 1%,23.741007,20.000000,28.571429,4646,4644,23.74 [20.0-28.57]
1,Top 5% vs Bottom 5%,15.217391,13.138686,17.266187,23227,23239,15.22 [13.14-17.27]
2,Top 10% vs Bottom 10%,11.594203,10.218978,13.571429,46427,46427,11.59 [10.22-13.57]
3,Top 25% vs Bottom 25%,8.029197,6.617647,8.759124,116067,116084,8.03 [6.62-8.76]


In [13]:
def twomils(dataframe, score):
    
    """
       Function to calculate the percentage of individuals with CRP
       levels ≥ 2 mg/L across different percentiles of the genetic score
    """
    col_name = f'SCORE1_SUM_{score}var'
    thresholds = {
    '1': np.percentile(dataframe[col_name], 1),
    '99': np.percentile(dataframe[col_name], 99),
    '5': np.percentile(dataframe[col_name], 5),
    '95': np.percentile(dataframe[col_name], 95),
    '10': np.percentile(dataframe[col_name], 10),
    '90': np.percentile(dataframe[col_name], 90),
    '25': np.percentile(dataframe[col_name], 25),
    '75': np.percentile(dataframe[col_name], 75)
    }

    # Define comparisons
    comparisons = [
        ('Top 25%', dataframe[dataframe[col_name] >= thresholds['75']]['CRP']),
         ('Bottom 25%', dataframe[dataframe[col_name] <= thresholds['25']]['CRP']),
        ('Top 10%', dataframe[dataframe[col_name] >= thresholds['90']]['CRP']),
        
        ('Bottom 10%', dataframe[dataframe[col_name] <= thresholds['10']]['CRP']),
        ('Top 5%', dataframe[dataframe[col_name] >= thresholds['95']]['CRP']),
        ('Bottom 5%', dataframe[dataframe[col_name] <= thresholds['5']]['CRP']),
        ('Top 1%', dataframe[dataframe[col_name] >= thresholds['99']]['CRP']),
        ('Bottom 1%', dataframe[dataframe[col_name] <= thresholds['1']]['CRP'])
        
    ]
    
    results = []
    for label, crp in comparisons:
        if len(crp)  > 0:
            results.append({
                'Comparison': f'{label}',
                '>=2 mg/l' : np.round(((sum(crp >=2) /len(crp)) * 100),2),
                "Median": np.round(crp.median(),2),
                'percentile_25' :np.round(crp.quantile(0.25),2),
                'percentile_75' : np.round(crp.quantile(0.75),2)
            })
            
    return results

In [14]:
descr_12 = pd.DataFrame(twomils(df_,'12'))
descr_12['Estimate']= descr_12.Median.astype(str) + ' [' + descr_12.percentile_25.astype(str) + '-'+ descr_12.percentile_75.astype(str) + ']'

In [15]:
descr_12

,Comparison,>=2 mg/l,Median,percentile_25,percentile_75,Estimate
0,Top 25%,36.48,1.37,0.68,2.84,1.37 [0.68-2.84]
1,Bottom 25%,33.69,1.26,0.62,2.64,1.26 [0.62-2.64]
2,Top 10%,36.76,1.38,0.69,2.87,1.38 [0.69-2.87]
3,Bottom 10%,32.53,1.22,0.61,2.55,1.22 [0.61-2.55]
4,Top 5%,36.92,1.38,0.69,2.88,1.38 [0.69-2.88]
5,Bottom 5%,31.40,1.17,0.58,2.46,1.17 [0.58-2.46]
6,Top 1%,36.85,1.39,0.70,2.86,1.39 [0.7-2.86]
7,Bottom 1%,28.49,1.06,0.51,2.26,1.06 [0.51-2.26]


In [ ]:
### Repeat for the two other genetic scores. 
res_8var = decrease_median(df_,'8')

descr_8 = pd.DataFrame(twomils(df_,'8'))
descr_8['Estimate']= descr_8.Median.astype(str) +\
' [' + descr_8.percentile_25.astype(str) + '-'+ descr_8.percentile_75.astype(str) + ']'descr_3 = pd.DataFrame(twomils(df_,'3'))

In [ ]:
res_3var = decrease_median(df_,'3')

descr_3 = pd.DataFrame(twomils(df_,'3'))
descr_3['Estimate']= descr_3.Median.astype(str) +\
' [' + descr_3.percentile_25.astype(str) + '-'+ descr_3.percentile_75.astype(str) + ']'

### Per anceestry analysis

In [16]:
white = df_[df_.ancestry.astype(str).str.startswith('1')]
black = df_[df_.ancestry.astype(str).str.startswith('4')]
asian = df_[df_.ancestry.astype(str).str.startswith('3')]

In [17]:
res_12var_white = decrease_median(white,'12')
res_8var_white = decrease_median(white,'8')
res_3var_white = decrease_median(white,'3')

In [19]:
res_12var_white['median_estimate'] = np.round(res_12var_white['Percent Decrease in Median CRP'],2).astype(str) + ' [' \
+np.round(res_12var_white['95% CI Lower'],2).astype(str) +'-'+ np.round(res_12var_white['95% CI Upper'],2).astype(str) + ']'

In [21]:
res_12var_white

,Comparison,Percent Decrease in Median CRP,95% CI Lower,95% CI Upper,Top n,Bottom n,median_estimate
0,Top 1% vs Bottom 1%,13.043478,8.759124,18.248867,4424,4412,13.04 [8.76-18.25]
1,Top 5% vs Bottom 5%,10.869565,8.823529,13.571429,22327,21895,10.87 [8.82-13.57]
2,Top 10% vs Bottom 10%,10.144928,8.088235,11.510791,43838,43832,10.14 [8.09-11.51]
3,Top 25% vs Bottom 25%,7.299270,6.569343,8.695652,109578,109446,7.3 [6.57-8.7]


The analysis is repeated per ancestry and for different GRS using the same functions as above

In [ ]:
res_12var_black = decrease_median(black,'12')
res_8var_black = decrease_median(black,'8')
res_3var_black = decrease_median(black,'3')

In [ ]:
res_12var_asian = decrease_median(asian,'12')
res_8var_asian = decrease_median(asian,'8')
res_3var_asian = decrease_median(asian,'3')